In [3]:
import torchreid
from torchreid import models, utils

torchreid.models.show_avai_models()

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [9]:
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources="market1501",
    targets="market1501",
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

In [10]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
).cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [11]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [12]:
engine.run(
    save_dir="log/resnet50",
    max_epoch=60,
    eval_freq=10,
    print_freq=10,
    test_only=False
)

=> Start training


/home/mbahng/miniconda3/envs/slam/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


epoch: [1/60][10/404]	time 0.321 (0.569)	data 0.000 (0.047)	eta 3:49:42	loss 6.8014 (6.7442)	acc 3.1250 (0.6250)	lr 0.000300
epoch: [1/60][20/404]	time 0.322 (0.445)	data 0.000 (0.024)	eta 2:59:43	loss 6.5115 (6.8506)	acc 0.0000 (1.4062)	lr 0.000300
epoch: [1/60][30/404]	time 0.322 (0.404)	data 0.000 (0.016)	eta 2:43:05	loss 6.6646 (6.7681)	acc 0.0000 (1.1458)	lr 0.000300
epoch: [1/60][40/404]	time 0.323 (0.384)	data 0.000 (0.012)	eta 2:34:49	loss 6.6476 (6.7362)	acc 3.1250 (1.0938)	lr 0.000300
epoch: [1/60][50/404]	time 0.325 (0.372)	data 0.000 (0.010)	eta 2:29:52	loss 6.5109 (6.7158)	acc 3.1250 (0.9375)	lr 0.000300
epoch: [1/60][60/404]	time 0.325 (0.364)	data 0.000 (0.008)	eta 2:26:38	loss 6.4781 (6.6912)	acc 3.1250 (0.9896)	lr 0.000300
epoch: [1/60][70/404]	time 0.325 (0.358)	data 0.000 (0.007)	eta 2:24:21	loss 6.6163 (6.6755)	acc 0.0000 (0.8929)	lr 0.000300
epoch: [1/60][80/404]	time 0.326 (0.354)	data 0.000 (0.006)	eta 2:22:39	loss 6.5739 (6.6540)	acc 0.0000 (0.9766)	lr 0.000300


KeyboardInterrupt: 